# Predicting UCI's Madelon Data Set Using Feature Selection and KNN, DecisionTrees, and RandomForests

By Jordan Fritz

## Introduction

### Data

The small data set used for this project is UCI's Madelon data set. This synthetic data contains data points grouped into 32 different clusters which are placed at the vertices of a five dimensional hypercube. Each cluster is randomly assigned +1 or -1 and all data points in that cluster is then assigned this value. Along with these 5 informative features (vertices of the hypercube), 15 linear combinations of these features are also added, forming a set of 20 redundant and informative features. 480 additional "noise" features are added, creating a dataset of 500 features where only 20 are informative, and only 5 of these are the original 5 features.

The larger data set is set up the same way, however, there are 500 additional noise features thus creating a 1000 feature dataset with only 20 informative and 5 "original" features.

### Problem Statement

The challenge is to develop a series of models for two purposes:

1. for the purposes of identifying relevant features. 
2. for the purposes of generating predictions from the model. 

### Metric

The metric that will be used for determining train and test scores for this data set is the accuracy test score. Since this is a classification problem, the accuracy metric will be determined by simply calculating the percentage of datapoints the model labeled correctly.

### Solution

Using the smaller UCI dataset (4400 observations, 500 features), dimensionality reduction was used to reduce the dataset from 500 features to 20 informative features using an $R^{2}$ score. These 20 features were further reduced using numerous feature reduction techniques (`SelectKBest` (SKB), Recursive Feature Elimination (`RFE`), `SelectFromModel`(SFM), `SelectPercentile`). Using at most the entire 20 feature dataset, and at least a 5 feature dataset found from one of these reduction techniques, various models were fitted using a grid search with cross-validation. The best test score (0.883) was returned when using a `KNeighborsClassifier` and 20 features. The optimal test score using only 5 features (0.865) was achieved using the `RFE` features found while fitting a `RandomForestClassifier` and using these features in a `RandomForestClassifier` model. Both of these models were fit using a dataset of 2000 observations and tested on a dataset of 600 observations.

Using the large Madelon dataset (220000 observations, 1000 features), the above process was repeated to return the top 20 most informative features. These continued to be reduced using the methods described above. Using the model with the highest test score from the smaller dataset (`RandomForestClassifier` with `BaggingClassifier`) and features determined using the `RFE` of the `RandomForestClassifier`, a test score of 0.866 was accomplished using only 5 features and a train set and test set of size 21000 observations and 5000 observations, respectively.

## Feature Selection

##### Why $R^{2}$ works

For the following descriptions about how the features were reduced from 500 (or 1000) to 20, an $R^{2}$ score is calculated by first selecting a feature, fitting a regression model using the other 499 (or 999) features, and then returning an $R^{2}$ score depicting how close the other 499 features were to being able to predict the selected feature.

The reason this process works for selecting important features is because the non-informative features are randomly assigned noise features. This means that these features cannot be predicted using the other features, and therefore will have very low (negative) $R^{2}$ scores. On the other hand, the informative features have linear relationships with other informative features, and therefore can be accurately predicted using the other features in the dataset. This would return a high $R^{2}$ score.

#### UCI Madelon Dataset

In order to reduce 500 features down to the 20 informative features, a function was created named `calculate_r2_for_feature` and placed in the `__init__.py` text file.

In this function, the feature being explored becomes the target, and the other 499 features are used to determine this feature. Both a `KNeighborsRegressor` and a `DecisionTreeRegressor` are fit using a train set of the 499 features as the "X" value (predictors) and the chosen feature being the "y" value (target). A test set of similar data is used to return the $R^{2}$ score.

After looping through all 500 features in the dataframe, each feature is assigned its corresponding $R^{2}$ score calculated from the function explained above. The following are the top 25 $R^{2}$ scores sorted by the `DecisionTreeRegressor` from a random sample of 10% of the data (three random samples of 10% of the data were taken and all three are included in the table below).

In [1]:
import pandas as pd
r2_df = pd.read_csv('files/r2_dataframe')
r2_df = r2_df.rename(columns={'Unnamed: 0':'Feature'})
display(r2_df)

,Feature,sample0_DecisionTree,sample0_KNeighbors,sample1_DecisionTree,sample1_KNeighbors,sample2_DecisionTree,sample2_KNeighbors
0,153,0.967131,0.800221,0.970234,0.796493,0.972782,0.738970
1,128,0.964194,0.870559,0.933456,0.870018,0.947502,0.876675
2,442,0.954980,0.707660,0.943497,0.737102,0.934667,0.641258
3,475,0.954109,0.761391,0.919846,0.841325,0.963652,0.732014
4,493,0.952823,0.845834,0.936959,0.840637,0.869712,0.853006
5,472,0.951643,0.811475,0.946205,0.805065,0.912156,0.808914
6,64,0.944953,0.838523,0.954013,0.833933,0.966130,0.825389
7,105,0.942681,0.749974,0.944329,0.704346,0.900712,0.759920
8,336,0.940724,0.824993,0.948935,0.816537,0.926678,0.794521
9,453,0.940601,0.851535,0.905721,0.867498,0.929027,0.888501


From the table above, exactly 20 features have positive $R^{2}$ values for both models and all three samples, and all other features have negative $R^{2}$ values for both models and all three samples. This makes it extremely likely that these 20 features are the 20 informative features that were being searched for, and the rest of the analysis is done using only these 20 features.

After these 20 features were determined, `SelectKBest`, `RFE`, `SelectFromModel`, and `SelectPercentile` were used to furthefeature reduction.

**`SelectKBest`**: Selects and returns the top K features in order of importance. For this dataset, K = 5 returned the features 64, 128, 241, 336, and 475 when run on the full 2000 observation dataset. However, for a population size of 2000, a 90% confidence interval with a 1% margin of error requires only 1656 observations. When all populations between 1650 and 2000 are used with a step size of 50 to calculate the features, the following are the percentage of times that each feature appears in the top 5:

64: 100%

241: 100%

336: 100%

475: 100%

128: 50%

338: 37.5%

472: 12.5%

These findings somewhat mirror the findings when run on the full dataset, so the `SelectKBest` features were determined to be 64, 128, 241, 336, and 475.

**`RFE`**: Eliminates features based on which combination of `n_features_to_select` gives the highest test score for a particular model. For this analysis, `RFE` was used with `DecisionTreeClassifier`, `LogisiticRegression`, and `RandomForestClassifier` with `n_features_to_select` = 5. The features that were returned were as follows:

`DecisionTreeClassifier`: 48, 105, 338, 442, 475

`LogisticRegression`: 28, 128, 318, 442, 451

`RandomForestClassifier`: 48, 105, 318, 338, 475

**`SelectFromModel`**: Selects the features that are the most important for a specific model based on importance weights. This can be more or less than 5 features. For this analysis, the models used were `DecisionTreeClassifier`, `LogisiticRegression`, and `RandomForestClassifier`. The `SelectFromModel` returned the following features:

`DecisionTreeClassifier`: 28, 48, 105, 153, 338, 378, 442, 451, 453, 475

`LogisticRegression`: 28, 153, 318, 338, 433, 442, 451, 475

`RandomForestClassifier`: 48, 105, 128, 241, 281, 318, 338, 378, 433

**`SelectPercentile`**: Selects features according to a percentile of the highest scores. Used a percentile of 25 for this analysis, and the following features were returned: 475, 241, 336, 64, 128. These are returned in order of their appearances, and matches the `SelectKBest` model exactly.



A union of all of these features creates a list of features: 28, 48, 64, 105, 128, 153, 241, 318, 336, 338, 378, 433, 442, 451, 453, and 475. These features were then used to fit an Ordinary Least Squares (`OLS`) model and the p-values for each feature is returned with the null hypothesis being stated as "the feature is not important in determining the target variable". The majority of these features had a p-value less than $10^{-4}$, however five of the features had large p-values. The following were the features with large p-values their corresponding p-value:

28 - 0.8731824724915853

153 - 0.13095957920122445

318 - 0.8338883796872708

433 - 0.1825285610504542

451 - 0.6041125129060525

#### Larger Madelon Dataset

Using a similar process as outlined in the UCI Madelon Dataset,  first only 4000 observations were used to calculate $R^{2}$ values for all 1000 features. The table below shows the top 25 $R^{2}$ values. Once again, there are exactly 20 positive $R^{2}$ values, while the rest of the 980 features all have negative values.

In [2]:
big_r2_df = pd.read_csv('files/bigdata_r2')
big_r2_df = big_r2_df.rename(columns={'Unnamed: 0':'Feature'})
display(big_r2_df)

,Feature,sample0_DecisionTree,sample0_KNeighbors,sample1_DecisionTree,sample1_KNeighbors,sample2_DecisionTree,sample2_KNeighbors
0,639,0.927408,0.765546,0.928160,0.736336,0.865986,0.654629
1,956,0.886172,0.762094,0.901443,0.721654,0.873643,0.648066
2,867,0.819643,0.616547,0.772777,0.636942,0.695628,0.585044
3,336,0.781835,0.631073,0.752961,0.696783,0.672042,0.522090
4,920,0.749918,0.493926,0.694970,0.419661,0.662646,0.527689
5,701,0.745191,0.516612,0.780464,0.576887,0.670117,0.547489
6,269,0.739760,0.485213,0.708770,0.662076,0.837314,0.608524
7,504,0.723879,0.542038,0.482788,0.452808,0.437115,0.356188
8,341,0.711988,0.670392,0.778390,0.711253,0.747792,0.703786
9,395,0.704585,0.552216,0.727241,0.599737,0.746817,0.624806


After these 20 features were determined, the server could handle loading in 60000 observations of the 20 features, rather than the original 4000 observations of 1000 features. 

Using these 20 features, SelectKBest, RFE, SelectFromModel, and SelectPercentile were used to further feature reduction.

**`SelectKBest`**: Using three different samples of 20000 observations and K = 5, the features 269, 341, 681, 701, and 920 were selected for all three samples.

**`RFE`**: For this analysis, `RFE` was used with `DecisionTreeClassifier`, `LogisiticRegression`, and `RandomForestClassifier` with `n_features_to_select` = 5. This model was run on only one of the 20000 observation samples. The features that were returned were as follows:

`DecisionTreeClassifier`: 269, 308, 724, 769, 829

`LogisticRegression`: 269, 504, 681, 829, 920

`RandomForestClassifier`: 269, 308, 395, 808, 920

**`SelectFromModel`**: For this analysis, the models used were `DecisionTreeClassifier`, `LogisiticRegression`, and `RandomForestClassifier`. This model was run on only one of the 20000 observation samples. The `SelectFromModel` returned the following features:

`DecisionTreeClassifier`: 269, 308, 681, 724, 736, 769, 808, 829, 920

`LogisticRegression`: 257, 269, 308, 341, 504, 681, 701, 769, 808, 829, 920

`RandomForestClassifier`: 269, 308, 395, 504, 681, 724, 769, 808, 920, 956

**`SelectPercentile`**: Used a percentile of 25 for this analysis and a single sample of 20000 observations. The following features were returned: 269, 701, 681, 920, 341. These are returned in order of their appearances, and they match the `SelectKBest` model exactly.



A union of all of these features creates a list of features: 257, 269, 308, 341, 504, 681, 701, 724, 736, 769, 808, 829, and 920.

## Benchmarking

#### UCI Madelon Dataset

Benchmarking was accomplished using naive fits for four unique models using raw data from the smaller UCI Madelon data set of 2000 train observations and 600 test. Only the 20 features found from the $R^{2}$ analysis were used. The four models used for benchmarking were `KNeighborsClassifier`, `DecisionTreeClassifier`, `RandomForestClassifier`, and `LogisticRegression`. The train and test scores were as follows:

In [3]:
raw_scores = pd.read_csv('files/raw_benchmark_scores')
raw_scores = raw_scores.rename(columns = {'Unnamed: 0': ''})
display(raw_scores)

,,KNeighborsClassifier,DecisionTreeClassifier,RandomForestClassifier,LogisticRegression
0,Train Score,0.838333,1.000000,0.976667,0.655000
1,Test Score,0.744444,0.722222,0.683333,0.522222


From the above table, the highest naive fit benchmark score occurs in the `KNeighborsClassifier` where the score is 0.744.

#### Larger Madelon Dataset

Benchmarking was accomplished using naive fits for four unique models using both raw and scaled data from the larger Madelon data set of 20000 train observations and 2000 test. Only the 20 features found from the $R^{2}$ analysis were used. The four models used for benchmarking were `KNeighborsClassifier`, `DecisionTreeClassifier`, `RandomForestClassifier`, and `LogisticRegression`. The train and test scores were as follows:

In [4]:
raw_scores_bigdata = pd.read_csv('files/raw_benchmark_bigdata')
raw_scores_bigdata = raw_scores_bigdata.rename(columns = {'Unnamed: 0': ''})
display(raw_scores_bigdata)

,,KNeighborsClassifier,DecisionTreeClassifier,RandomForestClassifier,LogisticRegression
0,Train Score,0.843000,1.0,0.987333,0.591167
1,Test Score,0.796667,0.7,0.760000,0.571667


From the table above, the highest benchmark score is 0.797 and is achieved using the `KNeighborsClassifier`.

## Model Selection and Results

#### UCI Madelon Dataset

Using various combinations of features and hyperparameters, the three models that were used were `KNeighborsClassifier`, `DecisionTreeClassifier`, and `RandomForestClassifier`. The `DecisionTreeClassifier` and the `RandomForestClassifier` were used in combination with a `BaggingClassifier` as well for some of its fits. The scores below represent various models and their fits using particular features and hyperparameters. All hyperparameters were determined using `GridSearchCV`, and all models were fit using a train set of 2000 observations and tested using a test set of 600 observations. All data was scaled prior to training and testing the models.

###### 1. `KNeighborsClassifier(algorithm = 'auto', n_neighbors = 6, weights = 'distance')` 
- 20 features: `[28, 48, 64, 105, 128, 153, 241, 281, 318, 336, 338, 378, 433, 442, 451, 453, 455, 472, 475, 493]` 
- *Train score = 1.0* | *Test score = 0.883*

###### 2. `KNeighborsClassifier(algorithm = 'auto', n_neighbors = 17, weights = 'distance')`
- 5 features selected using `SelectKBest`: `[64, 128, 241, 336, 475]` 
- *Train score = 1.0* | *Test score = 0.728*

###### 3. `DecisionTreeClassifier(criterion = 'entropy', max_depth = None, max_features = 0.01, min_samples_split = 0.01, random_state = 42)` 
- 5 features selected using `SelectKBest`: `[64, 128, 241, 336, 475]`
- *Train score = 1.0* | *Test score = 0.632*

---
- `BaggingClassifier(model, n_estimators = 15, max_features = 5, random_state = 42)`
- 5 features selected using `SelectKBest`: `[64, 128, 241, 336, 475]`
- *Train score = 0.860* | *Test score = 0.708*

---
- `BaggingClassifier(model, n_estimators = 40, max_features = 5, random_state = 42)`
- 5 features selected using `RFE` fit on `DecisionTreeClassifier`: `[48, 105, 338, 442, 475]`
- *Train score = 0.937* | *Test score = 0.835*

###### 4. `DecisionTreeClassifier(criterion = 'gini', max_depth = None, max_features = 0.01, min_samples_split = 0.01, random_state = 42)`
- 5 features selected using `RFE` fit on `DecisionTreeClassifier`: `[48, 105, 338, 442, 475]`
- *Train score = 0.857* | *Test score = 0.717*

---
- 10 features selected using `SelectFromModel` fit on `DecisionTreeClassifier`: `[28, 48, 105, 153, 338, 378, 442, 451, 453, 475]`
- *Train score = 0.865* | *Test score = 0.752*

###### 5. `RandomForestClassifier(criterion = 'entropy', max_depth = None, max_features = 'auto', n_estimators = 100, random_state = 42)`
- 5 features selected using `RFE` fit on `RandomForestClassifier`: `[105, 241, 318, 338, 378]`
- *Train score = 1.0* | *Test score = 0.865*

---
- `BaggingClassifier(model, max_features = 5, n_estimators = 45, max_samples = 0.8, random_state = 42)`
- 5 featues selected using `RFE` fit on `RandomForestClassifier`: `[105, 241, 318, 338, 378]`
- *Train score = 0.978* | *Test score = 0.850*

###### 6.  `RandomForestClassifier(criterion = 'entropy', max_depth = None, max_features = 'auto', n_estimators = 350, random_state = 42)`
- 11 features selected following the p-value test: `[48, 64, 105, 128, 241, 336, 338, 378, 442, 453, 475]`
- *Train score = 1.0* | *Test score = 0.87*

The highest test score using only 5 features was achieved using the `RandomForestClassifier` in model 5. The higest test score overall was achieved using all 20 features on the `KNeighborsClassifier` in model 1.

#### Larger Madelon Dataset

Using the information collected above, high scoring models from the smaller UCI Madelon dataset were used on the large dataset in order to achieve high test scores. The scores below represent various models and their fits using particular features and hyperparameters. All hyperparameters were determined using GridSearchCV for the smaller dataset and were not re-calibrated using the larger data set. All models were fit using a train set of 21000 observations and tested using three different test sets of 5000 observations. These test scores were then averaged out to obtain the final test score. All data was scaled prior to training and testing the models.

###### 1. `KNeighborsClassifier(algorithm = 'auto', n_neighbors = 6, weights = 'distance')` 
- 20 features: `[257, 269, 308, 315, 336, 341, 395, 504, 526, 639, 681, 701, 724, 736, 769, 808, 829, 867, 920, 956]` 
- *Train score = 1.0* | *Test score = 0.891*

###### 2. `RandomForestClassifier(criterion = 'entropy', max_depth = None, max_features = 'auto', n_estimators = 100, random_state = 42)`
- 5 features selected using `RFE` fit on `RandomForestClassifier`: `[269, 308, 395, 808, 920]`
- *Train score = 1.0* | *Test score = 0.802*

---
- `BaggingClassifier(model, max_samples = .8, max_features = 5, random_state=42)`
- 5 features selected using `RFE` fit on `RandomForestClassifier`: `[269, 308, 395, 808, 920]`
- *Train score = 0.963* | *Test score = 0.866*

## Conclusion